In [124]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz
!tar xf spark-3.2.0-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [125]:
%cd /content/drive/My Drive/Emory MSBA/Machine Learning II/HW/

/content/drive/My Drive/Emory MSBA/Machine Learning II/HW


In [126]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.clustering import KMeans
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.feature import ElementwiseProduct
from pyspark.mllib.stat import Statistics
import pandas as pd




---


**Steps 1 + 2:**

In [127]:
# Load the data from the delimited file "train.csv" into the rdd rawdata
rawdata = spark.read.csv("/content/drive/My Drive/Emory MSBA/Machine Learning II/HW/train.csv", inferSchema= True, header=True)
rawdata.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [128]:
# print first 10 rows
rawdata.show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [129]:
rawdata.describe()

DataFrame[summary: string, PassengerId: string, Survived: string, Pclass: string, Name: string, Sex: string, Age: string, SibSp: string, Parch: string, Ticket: string, Fare: string, Cabin: string, Embarked: string]

In [130]:
# summary statistics
rawdata.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [132]:
# 10 most freq values in DESC
rawdata.createOrReplaceTempView("train")
spark.sql("SELECT PassengerId, COUNT(PassengerID) AS `freq` FROM train GROUP BY PassengerID ORDER BY `freq` DESC LIMIT 10;").show()

+-----------+----+
|PassengerId|freq|
+-----------+----+
|        148|   1|
|        463|   1|
|        471|   1|
|        496|   1|
|        833|   1|
|        243|   1|
|        392|   1|
|        540|   1|
|        623|   1|
|        737|   1|
+-----------+----+



In [133]:
spark.sql("SELECT Survived, COUNT(Survived) AS `freq` FROM train GROUP BY Survived ORDER BY `freq` DESC LIMIT 10;").show()

+--------+----+
|Survived|freq|
+--------+----+
|       0| 549|
|       1| 342|
+--------+----+



In [134]:
spark.sql("SELECT Pclass, COUNT(Pclass) AS `freq` FROM train GROUP BY Pclass ORDER BY `freq` DESC LIMIT 10;").show()

+------+----+
|Pclass|freq|
+------+----+
|     3| 491|
|     1| 216|
|     2| 184|
+------+----+



In [135]:
spark.sql("SELECT Sex, COUNT(Sex) AS `freq` FROM train GROUP BY Sex ORDER BY `freq` DESC LIMIT 10;").show()

+------+----+
|   Sex|freq|
+------+----+
|  male| 577|
|female| 314|
+------+----+



In [136]:
spark.sql("SELECT Age, COUNT(Age) AS `freq` FROM train GROUP BY Age ORDER BY `freq` DESC LIMIT 10;").show()

+----+----+
| Age|freq|
+----+----+
|24.0|  30|
|22.0|  27|
|18.0|  26|
|19.0|  25|
|28.0|  25|
|30.0|  25|
|21.0|  24|
|25.0|  23|
|36.0|  22|
|29.0|  20|
+----+----+



In [137]:
spark.sql("SELECT SibSp, COUNT(SibSp) AS `freq` FROM train GROUP BY SibSp ORDER BY `freq` DESC LIMIT 10;").show()

+-----+----+
|SibSp|freq|
+-----+----+
|    0| 608|
|    1| 209|
|    2|  28|
|    4|  18|
|    3|  16|
|    8|   7|
|    5|   5|
+-----+----+



In [138]:
spark.sql("SELECT Parch, COUNT(Parch) AS `freq` FROM train GROUP BY Parch ORDER BY `freq` DESC LIMIT 10;").show()

+-----+----+
|Parch|freq|
+-----+----+
|    0| 678|
|    1| 118|
|    2|  80|
|    3|   5|
|    5|   5|
|    4|   4|
|    6|   1|
+-----+----+



In [139]:
spark.sql("SELECT Fare, COUNT(Fare) AS `freq` FROM train GROUP BY Fare ORDER BY `freq` DESC LIMIT 10;").show()

+------+----+
|  Fare|freq|
+------+----+
|  8.05|  43|
|  13.0|  42|
|7.8958|  38|
|  7.75|  34|
|  26.0|  31|
|  10.5|  24|
| 7.925|  18|
| 7.775|  16|
|7.2292|  15|
|   0.0|  15|
+------+----+



In [140]:
spark.sql("SELECT Cabin, COUNT(Cabin) AS `freq` FROM train GROUP BY Cabin ORDER BY `freq` DESC LIMIT 10;").show()

+-----------+----+
|      Cabin|freq|
+-----------+----+
|    B96 B98|   4|
|         G6|   4|
|C23 C25 C27|   4|
|         F2|   3|
|    C22 C26|   3|
|          D|   3|
|       E101|   3|
|        F33|   3|
|        C65|   2|
|B51 B53 B55|   2|
+-----------+----+



In [141]:
spark.sql("SELECT Embarked, COUNT(Embarked) AS `freq` FROM train GROUP BY Embarked ORDER BY `freq` DESC LIMIT 10;").show()

+--------+----+
|Embarked|freq|
+--------+----+
|       S| 644|
|       C| 168|
|       Q|  77|
|    null|   0|
+--------+----+



In [142]:
# Based on above results, I would keep all columns
# as features except: Cabin and PassengerID
# for the Logistic Regression Model because Cabin doesn't have enough populated 
# bins or categories to properly effect the logistic regression model while reducing
# the dimensionality of the data set.

---
**Step 3**


In [181]:
# Drop Cabin from df
columns_to_drop = ['PassengerID', 'Cabin', 'Name', 'Ticket']
df = rawdata.drop(*columns_to_drop)

In [182]:
df.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [183]:
# cast to double
from typing import cast
df = df.withColumn('Survived', df['Survived'].cast('double'))
df = df.withColumn('Pclass', df['Pclass'].cast('string'))
df = df.withColumn('SibSp', df['SibSp'].cast('double'))
df = df.withColumn('Parch', df['Parch'].cast('double'))
df = df.withColumn('Fare', df['Fare'].cast('double'))

In [223]:
# new variable: AgeNA with 0=no age, 1=has age
from pyspark.sql import functions as f
df = df.withColumn('AgeNA',
                   f.when(df.Age.isNull(), 0).otherwise(1))

# convert int to double for new var
df = df.withColumn('AgeNA', df['AgeNA'].cast('double'))

df.show()

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|
+--------+------+------+----+-----+-----+-------+--------+-----+
|     0.0|     3|  male|22.0|  1.0|  0.0|   7.25|       S|  1.0|
|     1.0|     1|female|38.0|  1.0|  0.0|71.2833|       C|  1.0|
|     1.0|     3|female|26.0|  0.0|  0.0|  7.925|       S|  1.0|
|     1.0|     1|female|35.0|  1.0|  0.0|   53.1|       S|  1.0|
|     0.0|     3|  male|35.0|  0.0|  0.0|   8.05|       S|  1.0|
|     0.0|     3|  male|29.7|  0.0|  0.0| 8.4583|       Q|  1.0|
|     0.0|     1|  male|54.0|  0.0|  0.0|51.8625|       S|  1.0|
|     0.0|     3|  male| 2.0|  3.0|  1.0| 21.075|       S|  1.0|
|     1.0|     3|female|27.0|  0.0|  2.0|11.1333|       S|  1.0|
|     1.0|     2|female|14.0|  1.0|  0.0|30.0708|       C|  1.0|
|     1.0|     3|female| 4.0|  1.0|  1.0|   16.7|       S|  1.0|
|     1.0|     1|female|58.0|  0.0|  0.0|  26.55|       S|  1.0|
|     0.0|     3|  male|2

In [224]:
# replace all missing values in Age w/ mean
df = df.withColumn('Age',
                   f.when(df.Age.isNull(), 29.70).otherwise(df.Age))
df.show()

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|
+--------+------+------+----+-----+-----+-------+--------+-----+
|     0.0|     3|  male|22.0|  1.0|  0.0|   7.25|       S|  1.0|
|     1.0|     1|female|38.0|  1.0|  0.0|71.2833|       C|  1.0|
|     1.0|     3|female|26.0|  0.0|  0.0|  7.925|       S|  1.0|
|     1.0|     1|female|35.0|  1.0|  0.0|   53.1|       S|  1.0|
|     0.0|     3|  male|35.0|  0.0|  0.0|   8.05|       S|  1.0|
|     0.0|     3|  male|29.7|  0.0|  0.0| 8.4583|       Q|  1.0|
|     0.0|     1|  male|54.0|  0.0|  0.0|51.8625|       S|  1.0|
|     0.0|     3|  male| 2.0|  3.0|  1.0| 21.075|       S|  1.0|
|     1.0|     3|female|27.0|  0.0|  2.0|11.1333|       S|  1.0|
|     1.0|     2|female|14.0|  1.0|  0.0|30.0708|       C|  1.0|
|     1.0|     3|female| 4.0|  1.0|  1.0|   16.7|       S|  1.0|
|     1.0|     1|female|58.0|  0.0|  0.0|  26.55|       S|  1.0|
|     0.0|     3|  male|2

In [225]:
# print revised dataframe and summary statistics
print(df.show())
print(df.printSchema())
print(df.summary().show())

+--------+------+------+----+-----+-----+-------+--------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|AgeNA|
+--------+------+------+----+-----+-----+-------+--------+-----+
|     0.0|     3|  male|22.0|  1.0|  0.0|   7.25|       S|  1.0|
|     1.0|     1|female|38.0|  1.0|  0.0|71.2833|       C|  1.0|
|     1.0|     3|female|26.0|  0.0|  0.0|  7.925|       S|  1.0|
|     1.0|     1|female|35.0|  1.0|  0.0|   53.1|       S|  1.0|
|     0.0|     3|  male|35.0|  0.0|  0.0|   8.05|       S|  1.0|
|     0.0|     3|  male|29.7|  0.0|  0.0| 8.4583|       Q|  1.0|
|     0.0|     1|  male|54.0|  0.0|  0.0|51.8625|       S|  1.0|
|     0.0|     3|  male| 2.0|  3.0|  1.0| 21.075|       S|  1.0|
|     1.0|     3|female|27.0|  0.0|  2.0|11.1333|       S|  1.0|
|     1.0|     2|female|14.0|  1.0|  0.0|30.0708|       C|  1.0|
|     1.0|     3|female| 4.0|  1.0|  1.0|   16.7|       S|  1.0|
|     1.0|     1|female|58.0|  0.0|  0.0|  26.55|       S|  1.0|
|     0.0|     3|  male|2

---
**Step 4**


In [257]:
# import all pyspark funcitons
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression, LogisticRegressionSummary
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.feature import OneHotEncoder

In [289]:
# create indexers and encoders for categorical string variables
indexedcols = ["Pclass_encoder", "Sex_encoder", "Embarked_encoder",
               "Age", "SibSp", "Parch", "Fare", "AgeNA"]
#cols = ["Age", "SibSp", "Parch", "Fare", "AgeNA"]

si1 = StringIndexer(inputCol="Pclass",outputCol="Pclass_indexer")
si2 = StringIndexer(inputCol="Sex",outputCol="Sex_indexer")
si3 = StringIndexer(inputCol="Embarked",outputCol="Embarked_indexer")

# handle null values
si1.setHandleInvalid("keep")
si2.setHandleInvalid("keep")
si3.setHandleInvalid("keep")

# do same for one hot encoder
enc1 = OneHotEncoder(inputCol="Pclass_indexer", outputCol="Pclass_encoder")
enc2 = OneHotEncoder(inputCol="Sex_indexer", outputCol="Sex_encoder")
enc3 = OneHotEncoder(inputCol="Embarked_indexer", outputCol="Embarked_encoder")

---
**Step 5**


In [290]:
# assemble all feature columns into a feature vector in order to be used in the pipeline
va = VectorAssembler(inputCols = indexedcols, outputCol = 'features')



---
**Step 6**


In [291]:
# create the logistic regression model to be used in the pipeline.
lgr = LogisticRegression(featuresCol = 'features', labelCol="Survived")
steps = [si1, si2, si3, enc1, enc2, enc3, va, lgr]



---
**Step 7**


In [292]:
# Assemble pipeline
pl = Pipeline(stages=steps)



---
**Step 8**


In [293]:
# use a 70-30 split for training to test
# verify the size after split
temp = df.randomSplit([0.7, 0.3])
train =  temp[0]
validation = temp[1]

df.createOrReplaceTempView("temp")
spark.sql("SELECT COUNT(*) FROM temp").show()

train.createOrReplaceTempView("train")
spark.sql("SELECT COUNT(*) FROM train").show()

validation.createOrReplaceTempView("validation")
spark.sql("SELECT COUNT(*) FROM validation").show()

print("training: ", 621/891)
print("validation: ", 270/891)

+--------+
|count(1)|
+--------+
|     891|
+--------+

+--------+
|count(1)|
+--------+
|     618|
+--------+

+--------+
|count(1)|
+--------+
|     273|
+--------+

training:  0.696969696969697
validation:  0.30303030303030304




---
**Step 9**


In [294]:
train.printSchema()

root
 |-- Survived: double (nullable = true)
 |-- Pclass: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: double (nullable = true)
 |-- Parch: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)
 |-- AgeNA: double (nullable = false)



In [295]:
# Fit the model using the predefined pipeline on the training set.
plmodel = pl.fit(train)

In [296]:
# Use the fitted model for prediction on the test set.  
predictions = plmodel.transform(validation)

In [342]:
# Report Logistic Regression COEs
print(plmodel.stages[-1].coefficients)
print(plmodel.stages[-1].intercept)
#plmodel.coefficients()
#plmodel.intercept()

# interpret the obtained COEs
# As there is a 1 unit change in each of the variables, the likeliehood of classification as either
# surviving or not changes by the COE of each variable respectively.  And the intercept is the
# starting likeliehood of classification.  For example the first value of -.95 means that a 1 unit
# increase in PClass_encoder will result in -.95 likeliehood of "Survived".

[-0.9522940655470143,1.1202203308567524,0.16737568744430875,-1.5130201303959965,1.5130201303837827,-18.547153653300747,-17.954500904862474,-18.4079724557349,-0.036648522948215634,-0.30734289537766707,-0.16244770962657087,0.0019035557232868604,0.0]
19.68626994193703


In [345]:
type(predictions)

pyspark.sql.dataframe.DataFrame



---
**Step 10**


In [343]:
# Print the first 5 rows of the results. 
predictions.select("Survived", "prediction").show(5)

+--------+----------+
|Survived|prediction|
+--------+----------+
|     0.0|       1.0|
|     0.0|       1.0|
|     0.0|       0.0|
|     0.0|       1.0|
|     0.0|       0.0|
+--------+----------+
only showing top 5 rows



In [356]:
# Report the AUC for this model.
plmodel.stages[-1].summary.roc.show()
print("AUC: ", plmodel.stages[-1].summary.areaUnderROC)

+---+--------------------+
|FPR|                 TPR|
+---+--------------------+
|0.0|                 0.0|
|0.0|0.004098360655737705|
|0.0| 0.00819672131147541|
|0.0|0.012295081967213115|
|0.0| 0.01639344262295082|
|0.0| 0.02459016393442623|
|0.0|0.028688524590163935|
|0.0| 0.03278688524590164|
|0.0|0.036885245901639344|
|0.0|0.040983606557377046|
|0.0|0.045081967213114756|
|0.0| 0.04918032786885246|
|0.0| 0.05327868852459016|
|0.0| 0.05737704918032787|
|0.0| 0.06147540983606557|
|0.0| 0.06557377049180328|
|0.0| 0.06967213114754098|
|0.0| 0.07377049180327869|
|0.0|  0.0778688524590164|
|0.0| 0.08196721311475409|
+---+--------------------+
only showing top 20 rows

AUC:  0.8731864206189184


/content/spark-3.2.0-bin-hadoop2.7/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [353]:
plmodel.stages[-1].summary.areaUnderROC

0.8731864206189184